In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/beamforming-data-analysis/beamforming_dataset.csv


# Import necessary libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

2025-07-23 18:08:12.389378: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753294092.600664      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753294092.659090      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 1. Data Loading

In [3]:
df = pd.read_csv('/kaggle/input/beamforming-data-analysis/beamforming_dataset.csv')

In [4]:
df.head()

,sample_id,user_loc_hist_x_0,user_loc_hist_y_0,user_loc_hist_x_1,user_loc_hist_y_1,user_loc_hist_x_2,user_loc_hist_y_2,user_loc_hist_x_3,user_loc_hist_y_3,user_loc_hist_x_4,...,csi_beam_5,csi_beam_6,csi_beam_7,beam_hist_0,beam_hist_1,beam_hist_2,beam_hist_3,beam_hist_4,sinr,optimal_beam_index
0,0,51.857691,16.431856,62.919147,-26.694539,-94.865295,-1.516257,-17.190054,-95.728383,33.980550,...,2.964829,5.932241,6.349953,4,1,2,2,6,6.369092,2
1,1,-37.225273,-16.337037,-53.455520,-94.147930,43.411029,98.100884,7.226816,-29.459182,70.369568,...,2.459980,6.943596,3.087622,0,4,0,0,2,12.462178,1
2,2,77.031690,3.970814,-46.483865,-28.278262,87.036809,-12.240793,-65.956373,0.682597,21.902704,...,3.444585,1.728221,6.249143,2,5,0,0,1,26.177661,2
3,3,-17.785836,4.961503,-49.034502,-38.873527,99.192286,-19.290668,28.382459,-83.847260,88.615291,...,1.277260,2.789594,8.394630,7,3,6,7,0,16.474037,7
4,4,20.725721,6.003185,54.122171,92.838712,26.094274,99.988461,89.469675,26.114648,-55.987237,...,6.461580,7.412339,8.544736,1,6,1,3,3,19.418128,1


Here in the data we have different features:

1. num_samples: Total number of data points (e.g., 10,000 different time instances/scenarios).

2. num_beams: Number of discrete beam directions available (e.g., 8).

3. history_length: How many past time steps we consider for user location and beamforming history.

4. User Location History (user_loc_hist_x_i, user_loc_hist_y_i): Simulated as 2D (x, y) coordinates. history_length pairs of (x, y) values for each sample, representing the path the user took.

5. Channel State Information (CSI) (csi_beam_i):
- Simplified as num_beams individual values. Each value represents the "strength" or "gain" of the channel if that particular beam were used.
- Higher values generally imply better channel conditions for that beam.
- In reality, CSI is a complex matrix or vector of channel impulse responses, often involving complex numbers. This simplified version is for demonstration.

6. Beamforming History (beam_hist_i):

- history_length integer values, each representing the index of the beam that was used at a previous time step.

- Values are np.random.randint from 0 to num_beams-1.

7. SINR (sinr):

- A single floating-point value representing the Signal-to-Interference-plus-Noise Ratio.

- Simulated as np.random.uniform within a typical dB range.

8. Optimal Beam Index (optimal_beam_index):

- This is our target variable. For this synthetic dataset, we've implemented a very simplistic rule:

- It primarily picks the beam with the highest current CSI gain.

- It includes a small, arbitrary "location bias" (e.g., if the user's last X coordinate is high, it might slightly shift the optimal beam index). This rule is crucial: In a real-world scenario, this value would be derived from actual network optimization, measurements, or a more sophisticated simulator. Your ML model will learn to approximate this complex underlying function.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sample_id           10000 non-null  int64  
 1   user_loc_hist_x_0   10000 non-null  float64
 2   user_loc_hist_y_0   10000 non-null  float64
 3   user_loc_hist_x_1   10000 non-null  float64
 4   user_loc_hist_y_1   10000 non-null  float64
 5   user_loc_hist_x_2   10000 non-null  float64
 6   user_loc_hist_y_2   10000 non-null  float64
 7   user_loc_hist_x_3   10000 non-null  float64
 8   user_loc_hist_y_3   10000 non-null  float64
 9   user_loc_hist_x_4   10000 non-null  float64
 10  user_loc_hist_y_4   10000 non-null  float64
 11  csi_beam_0          10000 non-null  float64
 12  csi_beam_1          10000 non-null  float64
 13  csi_beam_2          10000 non-null  float64
 14  csi_beam_3          10000 non-null  float64
 15  csi_beam_4          10000 non-null  float64
 16  csi_b

In [6]:
df.describe()

,sample_id,user_loc_hist_x_0,user_loc_hist_y_0,user_loc_hist_x_1,user_loc_hist_y_1,user_loc_hist_x_2,user_loc_hist_y_2,user_loc_hist_x_3,user_loc_hist_y_3,user_loc_hist_x_4,...,csi_beam_5,csi_beam_6,csi_beam_7,beam_hist_0,beam_hist_1,beam_hist_2,beam_hist_3,beam_hist_4,sinr,optimal_beam_index
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.00000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4999.50000,0.325104,-0.302147,-0.016580,0.620732,0.929860,0.627001,-0.236695,-1.035627,-1.247463,...,5.075503,5.036890,5.038112,3.50690,3.50220,3.493600,3.508700,3.491500,17.421383,3.616000
std,2886.89568,58.191010,57.530205,57.303778,57.772163,57.707768,57.587878,57.382690,57.714308,57.524857,...,2.880924,2.871532,2.847330,2.29305,2.28397,2.276769,2.293829,2.290776,7.196477,2.282688
min,0.00000,-99.947542,-99.998059,-99.992984,-99.996790,-99.986027,-99.993389,-99.968288,-99.999993,-99.954957,...,0.100200,0.100046,0.100335,0.00000,0.00000,0.000000,0.000000,0.000000,5.003507,0.000000
25%,2499.75000,-50.677117,-50.190089,-48.674077,-49.725459,-48.912149,-49.190754,-49.287389,-51.424974,-51.360085,...,2.586787,2.504898,2.573640,2.00000,2.00000,2.000000,2.000000,1.000000,11.136612,2.000000
50%,4999.50000,0.320167,0.200420,-0.259913,0.875761,1.350589,0.912906,0.384411,-2.171292,-1.677622,...,5.098972,5.059064,5.026271,3.00000,4.00000,3.000000,4.000000,4.000000,17.356167,4.000000
75%,7499.25000,50.938478,49.278699,48.660853,50.601777,50.775882,50.152045,48.653145,49.016479,48.414975,...,7.564823,7.537030,7.520178,6.00000,5.00000,5.000000,6.000000,5.000000,23.575427,6.000000
max,9999.00000,99.964268,99.983350,99.971405,99.922727,99.988871,99.988461,99.990378,99.999262,99.996028,...,9.999913,9.998839,9.996120,7.00000,7.00000,7.000000,7.000000,7.000000,29.997548,7.000000


In [7]:
df.isnull().sum()

sample_id             0
user_loc_hist_x_0     0
user_loc_hist_y_0     0
user_loc_hist_x_1     0
user_loc_hist_y_1     0
user_loc_hist_x_2     0
user_loc_hist_y_2     0
user_loc_hist_x_3     0
user_loc_hist_y_3     0
user_loc_hist_x_4     0
user_loc_hist_y_4     0
csi_beam_0            0
csi_beam_1            0
csi_beam_2            0
csi_beam_3            0
csi_beam_4            0
csi_beam_5            0
csi_beam_6            0
csi_beam_7            0
beam_hist_0           0
beam_hist_1           0
beam_hist_2           0
beam_hist_3           0
beam_hist_4           0
sinr                  0
optimal_beam_index    0
dtype: int64

# EDA

# Feature Engineering

- Feature Scaling and Normalization
- ENcoding categorical features
- Structuring inputs for DL

# Model Selection and Training

For this task we can use different models.

- Supervised learning: This is a multi-class classification problem where you predict one of num_beams classes.
- Deep Learning+ Embeddings. It is for the high- dimensional CSI/ History

    - Autoencoders to reduce the dimensionality or learning compressed representations of high- dimensional embedding, then use this embedding as an input to the main classification model
    - KNN - For similarity matching. For defining distance metric between samples. KNN Directly classifies based on the majority class of its nearest neighbors in the feature space.

# Results

# Conclusion